## Revenue Generation in Uniswap

In [1]:
import os
import copy
import numpy as np
import pandas as pd
import time
import datetime
import math
import matplotlib.pyplot as plt

In [2]:
from uniswappy import *

In [3]:
user_nm = 'user0'
dai_amount = 10000
sys_amount = 10000

In [4]:
dai1 = ERC20("DAI", "0x111")
sys1 = ERC20("SYS", "0x09")
exchg_data = UniswapExchangeData(tkn0 = sys1, tkn1 = dai1, symbol="LP", address="0x011")

factory = UniswapFactory("SYS pool factory", "0x2")
lp = factory.deploy(exchg_data)
lp.add_liquidity(user_nm, sys_amount, dai_amount, sys_amount, dai_amount)
lp.summary()

Exchange SYS-DAI (LP)
Reserves: SYS = 10000, DAI = 10000
Liquidity: 10000.0 



In [5]:
LPQuote(False).get_amount_from_lp(lp, sys1, 1)

1.9969005990701798

### Liquidity leak test-batch #1

In [6]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1)
print('Pre-tests:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we are starting with {:.6f} LP token'.format(lp.liquidity_providers[user_nm]))

Exchange SYS-DAI (LP)
Reserves: SYS = 10000, DAI = 10000
Liquidity: 10000.0 

Pre-tests:
- 1 LP token is 1.996901 DAI
- we are starting with 10000.000000 LP token


#### Test 1: SwapDeposit

In [7]:
requested_liquidity_in = 500
sys_before_test = lp.reserve0
sys_settlement_amt = RebaseIndexToken().apply(lp, sys1, requested_liquidity_in) 
dep = SwapDeposit().apply(lp, sys1, user_nm, sys_settlement_amt)
get_deposit = LPQuote(False).get_amount_from_lp(lp, sys1, lp.last_liquidity_deposit)
lp.summary()

Exchange SYS-DAI (LP)
Reserves: SYS = 10973.646046907035, DAI = 10000.0
Liquidity: 10474.802870401676 



In [8]:
out = WithdrawSwap().apply(lp, sys1, user_nm, get_deposit)
lp.summary()

Exchange SYS-DAI (LP)
Reserves: SYS = 10002.724148007163, DAI = 10000.0
Liquidity: 10000.0 



In [9]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1)
sys_after_test = lp.reserve0
print('Post test #1:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we have {:.6f} total LP token'.format(lp.liquidity_providers[user_nm]))
print('- we made a profit of {:.6f} SYS post swapping'.format(sys_after_test-sys_before_test))

Exchange SYS-DAI (LP)
Reserves: SYS = 10002.724148007163, DAI = 10000.0
Liquidity: 10000.0 

Post test #1:
- 1 LP token is 1.996901 DAI
- we have 10000.000000 total LP token
- we made a profit of 2.724148 SYS post swapping


#### Test 2: Multiple WithdrawSwap / SwapDeposit

In [10]:
sys_before_test = lp.reserve0
N = 100
for k in range(N):
    requested_liquidity_out = 500
    sys_settlement_amt = RebaseIndexToken().apply(lp, sys1, requested_liquidity_in) 
    dep = SwapDeposit().apply(lp, sys1, user_nm, sys_settlement_amt)
    get_deposit = LPQuote(False).get_amount_from_lp(lp, sys1, lp.last_liquidity_deposit)
    out = WithdrawSwap().apply(lp, sys1, user_nm, get_deposit)

In [11]:
lp.summary()
dai_amt = LPQuote(False).get_amount_from_lp(lp, dai1, 1)
sys_after_test = lp.reserve0
print('Post test #2:')
print('- 1 LP token is {:.6f} DAI'.format(dai_amt))
print('- we have {:.6f} LP token remaining'.format(lp.liquidity_providers[user_nm]))
print('- we made a profit of {:.6f} SYS post swapping'.format(sys_after_test-sys_before_test))

Exchange SYS-DAI (LP)
Reserves: SYS = 10278.920460583293, DAI = 10000.0
Liquidity: 9999.999999999989 

Post test #2:
- 1 LP token is 1.996901 DAI
- we have 10000.000000 LP token remaining
- we made a profit of 276.196313 SYS post swapping


In [12]:
LPQuote(False).get_amount_from_lp(lp, sys1, 1)

2.052598242553353